# Markov Chain Model to Determine UFC Fighter Value

## Abstract
The UFC is the largest mixed martial arts (MMA) promotion in the world. Many analytical models for MMA fights focus on prediction algorithms to find an edge in sports betting. In this study we will instead implement a Markov chain model to find fighters who are important to the "network" of fights. Such fighters are important because they are reliable and provide stability in the inherently chaotic world of fight matchmaking. Decisionmakers may find this information useful when forming strategies for fighter pay, contracts, and events. Public and media attention is often focused on the stars of the sport, such as Conor McGregor or Khabib Nurmagomedov, who make many times over the compensation given to the average fighter. However, the UFC is also smart enough to give respectable compensation to some long-established veteran fighters known as "gatekeepers" i.e, fighters who are not necessarily in strong contention for the championship belt, but who nonetheless serve as a litmus test for newer contenders seeking the throne. In short, this model attempts to find and evaluate those "gatekeepers" and maybe find some that UFC leadership might have missed.

## Methodology
We can visualize UFC fights as a graph or network, with fighter nodes connected by edges that represent an event where those fighters fought eachother. Below is one such network, an undirected graph with node size based on degree and edges color-coded by weight class.

<img src="figures/graph.svg" alt="Graph" style="width: 100%"/>

However for this analysis we want to know the "path" a fighter might take through the UFC. A fighter must work their way through the ranks in a semi-ordered fashion, fighting better and better opponents if they wish to reach the top. So for our simple directed graph, an edge between two fighters B and C exists if there is a fighter A who fought fighter B at time = t1 and then fought fighter C at time = t2. A weight is assigned to each edge dependent on the number of occurences of that specific relationship as well as the degree of the source node.

<img src="figures/diagram.png" alt="Diagram" style="width: 250px"/>

We will then transform the edge table into a sparse matrix $P$ and find and rank nodes by probabilistic steady state $x_t$ based on the following formula:

$x_t = P^T \cdot x(t - 1)$

## Results

In [13]:
from IPython.display import HTML
HTML(filename='figures/tables.html')

,Description,Markov Chain Rating
0,Donald Cerrone,3.947935e+30
1,Jim Miller,3.664943e+30
2,Demian Maia,3.236610e+30
3,Ross Pearson,2.880601e+30
4,Nate Marquardt,2.861583e+30
5,Rafael Dos Anjos,2.807866e+30
6,Diego Sanchez,2.740686e+30
7,Charles Oliveira,2.662260e+30
8,Joseph Benavidez,2.654093e+30
9,Glover Teixeira,2.595952e+30


## Discussion
Overall, I'm happy with the results. It passes my personal "eyeball" test for who I consider to be "gatekeepers" within the UFC. The biggest obstacle I came across and the source of the most variance for this model is the formula for calculating edge weights. I tried to find a balance between number of parallel edges and the degree of the source node. Interestingly, when the formula is heavily tilted towards the degree of the source node, the pound-for-pound ranking became extremely biased towards heavyweights. This could be due to a variety of factors, such as the long history of the weightclass or the fact that it is easier to move up a weightclass than down.

## Code

In [22]:
# import required packages
import pandas as pd
from collections import defaultdict
import scipy as sp
import scipy.sparse as sparse
import numpy as np

In [23]:
# read fight data from CSV
fights = pd.read_csv('fights.csv')
fights.head()

,Unnamed: 0,W/L,Str,Td,Sub,Pass,Weight class,Method,Round,Time,date,location,event,fighter_0,fighter_1
0,0,win,68 25,1 0,0 0,3 0,HW,KO/TKO Punches,2,3:00,"May 16, 2020","Jacksonville, Florida, USA",UFC Fight Night: Overeem vs. Harris,Alistair Overeem,Walt Harris
1,1,win,84 90,1 0,0 0,1 0,WSW,S-DEC,3,5:00,"May 16, 2020","Jacksonville, Florida, USA",UFC Fight Night: Overeem vs. Harris,Claudia Gadelha,Angela Hill
2,2,win,79 80,1 0,0 0,0 0,FW,S-DEC,3,5:00,"May 16, 2020","Jacksonville, Florida, USA",UFC Fight Night: Overeem vs. Harris,Dan Ige,Edson Barboza
3,3,win,66 41,0 0,0 0,0 0,MW,U-DEC,3,5:00,"May 16, 2020","Jacksonville, Florida, USA",UFC Fight Night: Overeem vs. Harris,Krzysztof Jotko,Eryk Anders
4,4,win,101 92,0 2,0 0,0 1,FW,U-DEC,3,5:00,"May 16, 2020","Jacksonville, Florida, USA",UFC Fight Night: Overeem vs. Harris,Song Yadong,Marlon Vera


In [24]:
# build dataframe containing matches between two fighters 
# and duplicate data so fighter A --> fights --> fighter B 
# and fighter B --> fights --> fighter A both occur.
# Sort by date is important for constructed directed edges
# later.

all_fights = fights.copy()[['fighter_0', 'fighter_1', 'date']]

# optional code to filter by weightclass
# all_fights = fights.copy()[fights['Weight class'] == 'HW'][['fighter_0', 'fighter_1', 'date']]

all_fights['date'] = pd.to_datetime(all_fights['date'])
all_fights2 = all_fights.copy()
all_fights2.columns = ['fighter_1', 'fighter_0', 'date']
all_fights2 = all_fights2[['fighter_0', 'fighter_1', 'date']]
all_fights = pd.concat([all_fights, all_fights2])
all_fights = all_fights.sort_values(['date', 'fighter_0', 'fighter_1']).reset_index(drop = True)
all_fights.head()

,fighter_0,fighter_1,date
0,Art Jimmerson,Royce Gracie,1993-11-12
1,Gerard Gordeau,Kevin Rosier,1993-11-12
2,Gerard Gordeau,Royce Gracie,1993-11-12
3,Gerard Gordeau,Teila Tuli,1993-11-12
4,Jason DeLucia,Trent Jenkins,1993-11-12


In [25]:
# group dataframe by fighter so that they are matched
# with a list of all their opponents in chronological 
# order.

edge_df = pd.DataFrame(all_fights.groupby(['fighter_0']).agg(
    opponents = ('fighter_1', list)))
edge_df = edge_df[edge_df['opponents'].map(len) > 1].reset_index()

assert len(edge_df) > 0
edge_df.head()

,fighter_0,opponents
0,Aaron Phillips,"[Sam Sicilia, Matt Hobar]"
1,Aaron Riley,"[Robbie Lawler, Spencer Fisher, Jorge Gurgel, ..."
2,Aaron Rosa,"[Joey Beltran, Matt Lucas, James Te Huna]"
3,Aaron Simpson,"[Tim McKenzie, Ed Herman, Tom Lawlor, Chris Le..."
4,Abdul Razak Alhassan,"[Charlie Ward, Omari Akhmedov, Sabah Homasi, S..."


In [26]:
# construct a dictionary of edges where the key is the
# two nodes and the value is the sum of the chosen
# formula for calculating count which will be a component
# of the final edge weight

edge_dict = defaultdict(int)
for edge_list in edge_df['opponents']:
    for i in range(len(edge_list) - 1):
        for j in range(min(2, len(edge_list[i+1:]) - 1)):
            f0 = edge_list[i]
            f1 = edge_list[i + j + 1]
            c = 1 / (j + 1) ** 2
            edge_dict[(f0, f1)] += c

In [27]:
# construct directed edge table with degree of the
# source node and final weight calculation

all_edges = pd.DataFrame()
all_edges['fighter_0'] = [k[0] for k in edge_dict.keys()]
all_edges['fighter_1'] = [k[1] for k in edge_dict.keys()]
all_edges['count'] = [v for k,v in edge_dict.items()]

outdegrees = all_edges[['fighter_0', 'fighter_1']].groupby(['fighter_0'], as_index = False).count()
outdegrees.columns = ['fighter_0', 'outdegree']
all_edges = all_edges.merge(outdegrees, on = 'fighter_0', how = 'left')

all_edges['outdegree'] = 1 / all_edges['outdegree']
all_edges['weight'] = all_edges['count'] + 999 * all_edges['outdegree']
all_edges

,fighter_0,fighter_1,count,outdegree,weight
0,Robbie Lawler,Spencer Fisher,1.00,0.038462,39.423077
1,Robbie Lawler,Jorge Gurgel,0.25,0.038462,38.673077
2,Spencer Fisher,Jorge Gurgel,1.00,0.034483,35.448276
3,Spencer Fisher,Shane Nelson,0.25,0.034483,34.698276
4,Jorge Gurgel,Shane Nelson,1.25,0.166667,167.750000
...,...,...,...,...,...
12443,Douglas Silva de Andrade,Ernest Chavez,1.00,0.111111,112.000000
12444,Douglas Silva de Andrade,Phillipe Nover,0.25,0.111111,111.250000
12445,Ernest Chavez,Phillipe Nover,1.00,0.200000,200.800000
12446,Ernest Chavez,Renato Moicano,0.25,0.200000,200.050000


In [28]:
# build translating dictionaries to pair fighter name
# with their index in the roster. Their index will
# become their index in the matrix.

roster = sorted(list(set(all_edges['fighter_0'].to_list() + all_edges['fighter_1'].to_list())))
roster_dict = {roster[i] : i for i in range(len(roster))}
roster_trans = {i : roster[i] for i in range(len(roster))}
roster_codes = pd.DataFrame(roster)
roster_codes.columns = ['f']

In [29]:
# implement translating dictionaries above

all_edges['f0'] = all_edges['fighter_0'].apply(lambda x : roster_dict[x])
all_edges['f1'] = all_edges['fighter_1'].apply(lambda x : roster_dict[x])
all_edges

,fighter_0,fighter_1,count,outdegree,weight,f0,f1
0,Robbie Lawler,Spencer Fisher,1.00,0.038462,39.423077,1481,1630
1,Robbie Lawler,Jorge Gurgel,0.25,0.038462,38.673077,1481,907
2,Spencer Fisher,Jorge Gurgel,1.00,0.034483,35.448276,1630,907
3,Spencer Fisher,Shane Nelson,0.25,0.034483,34.698276,1630,1607
4,Jorge Gurgel,Shane Nelson,1.25,0.166667,167.750000,907,1607
...,...,...,...,...,...,...,...
12443,Douglas Silva de Andrade,Ernest Chavez,1.00,0.111111,112.000000,494,553
12444,Douglas Silva de Andrade,Phillipe Nover,0.25,0.111111,111.250000,494,1403
12445,Ernest Chavez,Phillipe Nover,1.00,0.200000,200.800000,553,1403
12446,Ernest Chavez,Renato Moicano,0.25,0.200000,200.050000,553,1441


In [30]:
# build scipy COO matrix object and set an initial vector
# with the probability of starting on any given node.

P = sparse.coo_matrix((all_edges['weight'], (all_edges['f0'], all_edges['f1'])), shape=(len(roster), len(roster)))
x0 = np.array([1 / len(all_edges['f0'].unique())] * len(roster))

In [33]:
# function for evaluating Markov Chain
def eval_markov_chain(P, x0, t_max):
    x = x0
    for t in range(t_max):
        x = P.T.dot(x)
    return x

# make Markov Chain calculation
T_MAX = 50
x = eval_markov_chain(P, x0, T_MAX)

# find median number of steps a fighter takes in the graph
outdegree_inverse = 1 / all_edges['outdegree']
degree_median = int(round(outdegree_inverse.median()))
T_MAX = degree_median

# do the thing
x = eval_markov_chain(P, x0, T_MAX)

# output
ranks = np.argsort(-x)
desc = roster_codes.iloc[ranks]['f']
rating = x[ranks]
top = pd.DataFrame({'Description': desc, 'Markov Chain Rating': rating}).reset_index(drop = True)
top.to_csv('top.csv')